<a href="https://colab.research.google.com/github/yvsim001/projekt/blob/main/tar_meinMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install apache-tvm --pre

In [ ]:
!pip install pyserial==3.5 tflite==2.1 Pillow==9.0 typing_extensions

In [ ]:
import tensorflow as tf
print(tf.__version__)


2.17.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade tflite


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: tflite
    Found existing installation: tflite 2.1.0
    Uninstalling tflite-2.1.0:
      Successfully uninstalled tflite-2.1.0


In [ ]:
import os
import numpy as np
import pathlib
import json
from PIL import Image
import tarfile

import tvm
from tvm import relay
from tvm.relay.backend import Executor, Runtime
from tvm.contrib.download import download_testdata
from tvm.micro import export_model_library_format
from tvm.relay.op.contrib import cmsisnn
from tvm.micro.testing.utils import create_header_file

In [ ]:


import numpy as np
from tensorflow.keras.datasets import mnist

# Charger les données MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Les données doivent rester en uint8, donc pas de normalisation
# Ajouter une dimension de canal pour que les images soient au format attendu (batch_size, height, width, channels)
x_train = np.expand_dims(x_train, axis=-1).astype(np.uint8)
x_test = np.expand_dims(x_test, axis=-1).astype(np.uint8)

# Vérifier les types de données
print(f"x_train dtype: {x_train.dtype}, y_train dtype: {y_train.dtype}")
print(f"x_test dtype: {x_test.dtype}, y_test dtype: {y_test.dtype}")

# Sauvegarder les données dans un fichier .npz
np.savez('/content/drive/MyDrive/trained_models/mnist_uint8.npz', x_test=x_test[0:100], y_test=y_test[0:100])

print("Fichier mnist_uint8.npz créé avec succès.")


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
x_train dtype: uint8, y_train dtype: uint8
x_test dtype: uint8, y_test dtype: uint8
Fichier mnist_uint8.npz créé avec succès.


In [ ]:
MODEL_URL = "/content/drive/MyDrive/trained_models/mnist_model_quant.tflite"
MODEL_NAME = "mnist_model_quant.tflite"
MODEL_PATH = MODEL_URL

tflite_model_buf = open(MODEL_PATH, "rb").read()
try:
    import tflite

    tflite_model = tflite.Model.GetRootAsModel(tflite_model_buf, 0)
except AttributeError:
    import tflite.Model

    tflite_model = tflite.Model.Model.GetRootAsModel(tflite_model_buf, 0)



input_shape = (1, 28, 28, 1)
INPUT_NAME = "input_1_uint8"

from tvm import relay ,transform
relay_mod, params = tvm.relay.frontend.from_tflite(
    tflite_model, shape_dict={INPUT_NAME: input_shape}, dtype_dict={INPUT_NAME: "uint8"}
)

# Print the Relay module to inspect the weights
print(relay_mod)

# Check the type and shape of the weight tensors
for name, param in params.items():
    print(f"Name: {name}, Type: {type(param)}, Shape: {param.shape}")

def @main(%input_1: Tensor[(1, 28, 28, 1), uint8] /* span=input_1:0:0 */, %v_param_1: Tensor[(3, 3, 1, 12), int8] /* span=sequential/conv2d/Conv2D:0:0 */, %v_param_2: Tensor[(12), int32] /* span=sequential/conv2d/BiasAdd;sequential/conv2d/Conv2D;sequential/conv2d/BiasAdd/ReadVariableOp/resource:0:0 */, %v_param_3: Tensor[(3, 3, 12, 32), int8] /* span=sequential/conv2d_1/Conv2D:0:0 */, %v_param_4: Tensor[(32), int32] /* span=sequential/conv2d_1/BiasAdd;sequential/conv2d_1/Conv2D;sequential/conv2d_1/BiasAdd/ReadVariableOp/resource:0:0 */, %v_param_5: Tensor[(3, 3, 32, 64), int8] /* span=sequential/conv2d_2/Conv2D:0:0 */, %v_param_6: Tensor[(64), int32] /* span=sequential/conv2d_2/BiasAdd;sequential/conv2d_2/Conv2D;sequential/conv2d_2/BiasAdd/ReadVariableOp/resource:0:0 */, %v_param_7: Tensor[(10, 64), int8] /* span=sequential/dense/MatMul:0:0 */, %v_param_8: Tensor[(10), int32] /* span=sequential/dense/BiasAdd/ReadVariableOp/resource:0:0 */, output_tensor_names=["Identity"]) {
  %0 = qnn

In [ ]:
version = tflite_model.Version()
print("Model Version: " + str(version))

Model Version: 3


In [ ]:
# We can use TVM native schedules or rely on the CMSIS-NN kernels using TVM Bring-Your-Own-Code (BYOC) capability.
USE_CMSIS_NN = True

# USMP (Unified Static Memory Planning) performs memory planning of all tensors holistically to achieve best memory utilization
DISABLE_USMP = False

# Use the C runtime (crt)
RUNTIME = Runtime("crt")

# We define the target by passing the board name to `tvm.target.target.micro`.
# If your board is not included in the supported models, you can define the target such as:
#TARGET = tvm.target.Target("c -keys=arm_cpu,cpu -mcpu=cortex-m4")
TARGET = tvm.target.target.micro("stm32l4r5zi")

# Use the AOT executor rather than graph or vm executors. Use unpacked API and C calling style.
EXECUTOR = tvm.relay.backend.Executor(
    "aot", {"unpacked-api": True, "interface-api": "c", "workspace-byte-alignment": 8}
)

# Now, we set the compilation configurations and compile the model for the target:
config = {"tir.disable_vectorize": True}
#if USE_CMSIS_NN:
#    config["relay.ext.cmsisnn.options"] = {"mcpu": TARGET.mcpu}
#if DISABLE_USMP:
#    config["tir.usmp.enable"] = False

with tvm.transform.PassContext(opt_level=3, config=config):
    if USE_CMSIS_NN:
        # When we are using CMSIS-NN, TVM searches for patterns in the
        # relay graph that it can offload to the CMSIS-NN kernels.
        relay_mod = cmsisnn.partition_for_cmsisnn(relay_mod, params, mcpu=TARGET.mcpu)
        lowered = tvm.relay.build(
        relay_mod, target=TARGET, params=params, runtime=RUNTIME, executor=EXECUTOR
    )
parameter_size = len(tvm.runtime.save_param_dict(lowered.get_params()))
print(f"Model parameter size: {parameter_size}")

# We need to pick a directory where our file will be saved.
# If running on Google Colab, we'll save everything in ``/root/tutorial`` (aka ``~/tutorial``)
# but you'll probably want to store it elsewhere if running locally.

BUILD_DIR = pathlib.Path("/content/drive/MyDrive/Rar_files")

BUILD_DIR.mkdir(exist_ok=True)

# Now, we export the model into a tar file:
TAR_PATH = pathlib.Path(BUILD_DIR) / "model_meinMNIST.tar"
export_model_library_format(lowered, TAR_PATH)

Model parameter size: 32


PosixPath('/content/drive/MyDrive/Rar_files/model_meinMNIST.tar')

In [ ]:
with tarfile.open(TAR_PATH, mode="a") as tar_file:
    SAMPLES_DIR = "samples"
    SAMPLE_NUMMER2_PATH = "/content/drive/MyDrive/RANDOM_IMAGE_FOR_TEST/Nummer_2.jpg"
    SAMPLE_NUMMER5_PATH = "/content/drive/MyDrive/RANDOM_IMAGE_FOR_TEST/Nummer_5.jpg"

    if os.path.exists(SAMPLE_NUMMER2_PATH):
        with open(SAMPLE_NUMMER2_PATH, 'rb') as f:
            try:
                img = Image.open(f).convert('RGB')
                create_header_file("nummer_2", np.asarray(img), SAMPLES_DIR, tar_file)
            except Exception as e:
                print(f"Error opening image: {e}")
    else:
        print(f"File not found: {SAMPLE_NUMMER2_PATH}")

    # Repeat the same process for SAMPLE_NUMMER5_PATH
    if os.path.exists(SAMPLE_NUMMER5_PATH):
        with open(SAMPLE_NUMMER5_PATH, 'rb') as f:
            try:
                img = Image.open(f).convert('RGB')
                create_header_file("nummer_5", np.asarray(img), SAMPLES_DIR, tar_file)
            except Exception as e:
                print(f"Error opening image: {e}")
    else:
        print(f"File not found: {SAMPLE_NUMMER5_PATH}")